In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../benchmark/processed_labels.csv')

In [6]:
df.head(2)

,FileName,Angry,Sad,Happy,Surprise,Fear,Disgust,Contempt,Neutral,Split_Set
0,MSP-PODCAST_0002_0033.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development
1,MSP-PODCAST_0002_0039.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development


In [7]:
df.Split_Set.value_counts(normalize=True), df.Split_Set.value_counts(normalize=False)

(Split_Set
 Train          0.726201
 Development    0.273799
 Name: proportion, dtype: float64,
 Split_Set
 Train          66992
 Development    25258
 Name: count, dtype: int64)

In [8]:
df_2 = pd.read_csv('/workspace/lucas.ueda/interspeech_ser/data/Labels/labels_consensus.csv')

In [9]:
df_2.head()

,FileName,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_0002_0033.wav,N,4.8,4.2,5.4,127,Female,Development
1,MSP-PODCAST_0002_0039.wav,N,4.0,4.2,4.2,127,Female,Development
2,MSP-PODCAST_0002_0051.wav,N,4.0,4.2,4.2,127,Female,Development
3,MSP-PODCAST_0002_0059.wav,X,4.0,3.8,4.0,128,Female,Development
4,MSP-PODCAST_0002_0061.wav,F,3.4,2.8,4.2,128,Female,Development


In [10]:
df_2.Split_Set.value_counts(normalize=True), df_2.Split_Set.value_counts(normalize=False)

(Split_Set
 Train          0.724998
 Development    0.275002
 Name: proportion, dtype: float64,
 Split_Set
 Train          84260
 Development    31961
 Name: count, dtype: int64)

# Lets take a look to df which are the ones with well known categorical label

In [11]:
df_full = df.merge(df_2[['FileName','SpkrID','Gender']], on ='FileName', how='left')
df_full.head()

,FileName,Angry,Sad,Happy,Surprise,Fear,Disgust,Contempt,Neutral,Split_Set,SpkrID,Gender
0,MSP-PODCAST_0002_0033.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development,127,Female
1,MSP-PODCAST_0002_0039.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development,127,Female
2,MSP-PODCAST_0002_0051.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development,127,Female
3,MSP-PODCAST_0002_0061.wav,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Development,128,Female
4,MSP-PODCAST_0003_0052.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Train,3,Male


Is there any spk equal in train and dev?


In [12]:
dev_spks = df_full[df_full['Split_Set'] == 'Development'].SpkrID.unique()
train_spks = df_full[df_full['Split_Set'] == 'Train'].SpkrID.unique()

In [13]:
equal = list(set(dev_spks) & set(train_spks))

len(equal)

0

What are the test partition? are all audios from Audios folder in df_full?

In [14]:
import os
total_files = os.listdir('/workspace/lucas.ueda/interspeech_ser/data/Audios')

In [15]:
len(total_files)

119421

In [16]:
len(total_files)

119421

In [17]:
tr_files = df_full[df_full['Split_Set'] == 'Train'].FileName.unique()

In [18]:
equal2 = list(set(total_files) & set(tr_files))

len(equal2), len(tr_files)

(66992, 66992)

In [19]:
test = pd.read_csv('./Categorical_test.csv')
test.shape

(3200, 2)

In [20]:
test_files = test['FileName'].unique()
equal3 = list(set(total_files) & set(test_files))

len(equal3), len(test_files)

(3200, 3200)

In [21]:
for file in test_files:
    if(file not in total_files):
        print('error')
        break

In [22]:
total_files[0]

'MSP-PODCAST_3085_0127.wav'

In [23]:
df_full.Gender.value_counts()

Gender
Male      52774
Female    39476
Name: count, dtype: int64

In [24]:
df_full.SpkrID.nunique()

2779

In [25]:
emotions = ["Angry", "Sad", "Happy", "Surprise", "Fear", "Disgust", "Contempt", "Neutral"]

In [26]:
for e in emotions:
    total = df_full[e].sum()/df_full.shape[0]
    print(f'Emotion {e} = {total}')

Emotion Angry = 0.13622764227642276
Emotion Sad = 0.09373441734417344
Emotion Happy = 0.24998373983739838
Emotion Surprise = 0.04265582655826558
Emotion Fear = 0.01567479674796748
Emotion Disgust = 0.021398373983739838
Emotion Contempt = 0.04286178861788618
Emotion Neutral = 0.39746341463414636


In [27]:
for e in emotions:
    total_male = df_full[(df_full[e] == 1)&(df_full['Gender'] == 'Male')].shape[0]
    total_female = df_full[(df_full[e] == 1)&(df_full['Gender'] == 'Female')].shape[0]
    print(f'Emotion {e} Male = {total_male}, Female = {total_female}')

Emotion Angry Male = 7948, Female = 4619
Emotion Sad Male = 4496, Female = 4151
Emotion Happy Male = 12678, Female = 10383
Emotion Surprise Male = 2264, Female = 1671
Emotion Fear Male = 639, Female = 807
Emotion Disgust Male = 1154, Female = 820
Emotion Contempt Male = 2160, Female = 1794
Emotion Neutral Male = 21435, Female = 15231


In [28]:
for e in emotions:
    total = df_full[(df_full[e] == 1)]['SpkrID'].nunique()
    print(f'Emotion {e} Unique spks = {total}')

Emotion Angry Unique spks = 1284
Emotion Sad Unique spks = 1576
Emotion Happy Unique spks = 2076
Emotion Surprise Unique spks = 1072
Emotion Fear Unique spks = 478
Emotion Disgust Unique spks = 582
Emotion Contempt Unique spks = 970
Emotion Neutral Unique spks = 2317


In [29]:
import IPython

In [48]:
df_full[df_full['Angry']==1].head(5)

,FileName,Angry,Sad,Happy,Surprise,Fear,Disgust,Contempt,Neutral,Split_Set,SpkrID,Gender,l,sr
16,MSP-PODCAST_0003_0285.wav,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Train,3,Male,8.158062,16000
26,MSP-PODCAST_0003_0435.wav,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Train,3,Male,7.217313,16000
27,MSP-PODCAST_0003_0448.wav,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Train,3,Male,7.371562,16000
43,MSP-PODCAST_0003_0630.wav,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Train,3,Male,10.687188,16000
51,MSP-PODCAST_0005_0065.wav,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Train,85,Male,5.930000,16000


In [50]:
IPython.display.Audio('/workspace/lucas.ueda/interspeech_ser/data/Audios/' + 'MSP-PODCAST_0003_0435.wav')

In [30]:
IPython.display.Audio('/workspace/lucas.ueda/interspeech_ser/data/Audios/' + df_full['FileName'].values[-70000])

In [31]:
ROOT = '/workspace/lucas.ueda/interspeech_ser/data/Audios/'

In [32]:
import librosa
def get_len(wpath):
    y, sr = librosa.load(wpath, sr = None)

    return len(y)/sr, sr

In [33]:
srs = []
lens = []

for w in df_full.FileName:
    wpath = ROOT + w
    l, sr = get_len(wpath)
    srs.append(sr)
    lens.append(l)

df_full['l'] = lens
df_full['sr'] = srs

In [34]:
df_full.sr.unique()

array([16000])

In [35]:
df_full.l.max(), df_full.l.min(), df_full.l.mean(), df_full.l.quantile(.9), df_full.l.quantile(.75), df_full.l.quantile(.95), df_full.l.quantile(.5)

(11.2036875, 1.9128125, 5.668917334688347, 9.344, 7.296, 10.112, 5.12)

In [36]:
for e in emotions:
    total = df_full[(df_full[e] == 1)]['l'].sum()/3600
    print(f'Emotion {e} Unique spks = {total}h')

Emotion Angry Unique spks = 19.435874652777777h
Emotion Sad Unique spks = 13.48744605902778h
Emotion Happy Unique spks = 36.89135880208333h
Emotion Surprise Unique spks = 6.190169010416667h
Emotion Fear Unique spks = 2.2463027430555558h
Emotion Disgust Unique spks = 3.3864180555555556h
Emotion Contempt Unique spks = 6.771249322916667h
Emotion Neutral Unique spks = 56.85718805555556h


In [38]:
for e in emotions:
    total = df_full[(df_full[e] == 1)&(df_full['Split_Set']=='Train')]['l'].sum()/3600
    print(f'Emotion {e} Unique spks = {total}h')

Emotion Angry Unique spks = 10.251892621527778h
Emotion Sad Unique spks = 9.850876579861112h
Emotion Happy Unique spks = 27.01922765625h
Emotion Surprise Unique spks = 4.722772274305556h
Emotion Fear Unique spks = 1.7561253472222222h
Emotion Disgust Unique spks = 2.415638871527778h
Emotion Contempt Unique spks = 4.244042934027777h
Emotion Neutral Unique spks = 45.58514303819445h


In [53]:
for e in emotions:
    total = df_full[(df_full[e] == 1)&(df_full['Split_Set']=='Train')]['l'].mean()
    std = df_full[(df_full[e] == 1)&(df_full['Split_Set']=='Train')]['l'].std()
    print(f'Emotion {e} Unique spks = {total} +- {std}')

Emotion Angry Unique spks = 5.4831100040855745 +- 2.1641841115036957
Emotion Sad Unique spks = 5.623716410957818 +- 2.2204582670914843
Emotion Happy Unique spks = 5.818581058951965 +- 2.3948454111453104
Emotion Surprise Unique spks = 5.767293143656717 +- 2.4737563032237064
Emotion Fear Unique spks = 5.644688616071429 +- 2.491702348628478
Emotion Disgust Unique spks = 6.072835151885474 +- 2.3782922301389404
Emotion Contempt Unique spks = 6.123669163326653 +- 2.4022749965095724
Emotion Neutral Unique spks = 5.611822143333448 +- 2.304787321570298


In [42]:
import numpy as np
df_full.groupby('SpkrID').agg({'l': np.sum}).reset_index().sort_values(by='l', ascending=True)

/tmp/ipykernel_169728/3656949885.py:2: FutureWarning: The provided callable <function sum at 0x704324b47760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  df_full.groupby('SpkrID').agg({'l': np.sum}).reset_index().sort_values(by='l', ascending=True)


,SpkrID,l
1052,1201,2.432000
1011,1101,2.560000
2594,3129,2.800000
2025,2485,2.816000
1929,2388,2.816000
...,...,...
1327,1519,4519.040000
2285,2789,5832.376000
62,97,6924.211500
420,480,8611.738250


In [51]:
import numpy as np
df_full.groupby('EmoClass').agg({'l': np.sum}).reset_index().sort_values(by='l', ascending=True)

KeyError: 'EmoClass'

In [1]:

!ls ../data_tmp/

hubert_xlarge  wav2vec2_large_robust  wavlm_base_plus_sv
roberta_large  wav2vec2_xls_2b	      wavlm_large


In [3]:
import os
os.listdir('../data_tmp/wav2vec2_large_robust')[:3]

['MSP-PODCAST_0656_0234.pt',
 'MSP-PODCAST_3571_0440_0001.pt',
 'MSP-PODCAST_0116_0452.pt']

In [5]:
import torch
torch.load('../data_tmp/wav2vec2_large_robust/MSP-PODCAST_0656_0234.pt').shape

torch.Size([371, 1024])